In [ ]:
#Installing required Libraries wherever the code is being executed - run only if needed
#!pip install soundex
!pip install jellyfish

In [3]:
#Importing required Libraries
import pandas as pd 
import jellyfish
import collections
import numpy as np

#import soundex as sdx
#import fuzzy
#import pandasql as psql

In [4]:
# Please remember to navigate to "Shared with me" on your Gdrive home sceen
# Right click on the "HNY_Project" (the primary folder for this project) and select "Add shortcut to Drive"
# This way the below code can find a link to the Project dr folder through your own drive.


# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


# Pre-Preparation of the Data

Ideally after this pre-preperation we should write this data to a Database so all the processes can read the same data and dont have to repeat these steps.

In [283]:
# Reading both the Tesseract output files
tess_out_bk_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Tesseract Output/BK_1850_final_output_tesseract.csv', index_col=0)
tess_out_mn_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Tesseract Output/MN_1850_final_output_tesseract.csv', index_col=0)

In [284]:
# Adding a identifier column based on the dataset they origin from
tess_out_bk_1850["dataset_id"] = 'BK_1850'
tess_out_mn_1850["dataset_id"] = 'MN_1850'

# Appending both dataframes 
frames = [tess_out_bk_1850,tess_out_mn_1850]
tess_out_1850= pd.concat(frames).reset_index(drop=True)

In [285]:
# Creating a column with a 6-digit unique ID per row based on the row index
# Warning: This is being done with no verification of duplication
tess_out_1850['cd_unique_id'] = (tess_out_1850.index+1)+100000

In [286]:
# Reading both the Adress Standardized files
#stand_out_bk_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/BK_1850_with_address2.csv')
#stand_out_mn_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/mn_1850_for_rerun.csv')

In [287]:
df = tess_out_1850

In [288]:
# Standardizing Column names
df.columns = df.columns.str.replace(' ', '_') # replacing spaces (' ') with  underscore.
df.columns = df.columns.str.replace('-', '_') # replacing hyphens ('-') with underscore
df.columns= df.columns.str.lower() # converting column names to lower case

## Removing White Spaces and creating copy columns which we'll standardize

In [289]:
# Using Strip function to remove preceeding and following whitespaces
df['f_full_name'] = df['full_name'].str.strip()
df['f_first_name'] = df['first_name'].str.strip()
df['f_last_name'] = df['last_name'].str.strip()
df['f_middle_name'] = df['middle_name'].str.strip()
df['f_title'] = df['title']

# Summary of Approach and Method

The Approach we will take is to keep the original columns as is and work on copy of those columns in the data frame, that way at any point we have the original and the standardized columsn to compare side-by-side 

# Flagging Rows of specific nature

### Flagging certain columns as 'junk_entries'

In [290]:
df.loc[df['f_full_name'].str.contains("BROOKLYN", na=False),'junk_entries'] = 1
df['junk_entries'] = df['junk_entries'].fillna(0)

### Flagging rows with (refused) entries

In [291]:
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("\(refused", na=False),'f_refused_flag'] = 1
df['f_refused_flag'] = df['f_refused_flag'].fillna(0)

In [292]:
df['f_first_name'] =  df['f_first_name'].astype(str).str.strip().str.replace('\(refused\)','(refused)')
df['f_last_name'] =  df['f_last_name'].astype(str).str.strip().str.replace('\(refused\)','(refused)')
df['f_middle_name'] =  df['f_middle_name'].astype(str).str.strip().str.replace('\(refused\)','(refused)')

### Flagging rows '(col'd)' into a column race_info_avl

In [293]:
# Flagging
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("\(col'd\)", na=False),'f_race_info_avl'] = 1
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("\(col'd", na=False),'f_race_info_avl'] = 1
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("col'd\)", na=False),'f_race_info_avl'] = 1
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("\(cold\)", na=False),'f_race_info_avl'] = 1
df['f_race_info_avl'] = df['f_race_info_avl'].fillna(0)

In [294]:
# Removing after the Flagging
pattern = '|'.join(['\(col\'d\)', '\(col\'d', 'col\'d\)', '\(cold\)'])

df['f_first_name'] = df['f_first_name'].astype(str).str.strip().str.replace(pattern, '')
df['f_last_name'] = df['f_last_name'].astype(str).str.strip().str.replace(pattern, '')
df['f_middle_name'] = df['f_middle_name'].astype(str).str.strip().str.replace(pattern, '')

### Flagging '&' based entries which can be companies

In [295]:
# Flagging
df.loc[df['f_full_name'].str.contains("&", na=False),'f_non_person_flag'] = 1
df['f_non_person_flag'] = df['f_non_person_flag'].fillna(0)

# Particulars

### Particular Names

In [296]:
# Crawford
df.loc[df['f_full_name'].str.contains("Craw ford@John \(Rev", na=False),'f_first_name'] = df.loc[df['f_full_name'].str.contains("Craw ford@John \(Rev", na=False),'f_first_name'].str.replace('ford\@','')
df.loc[df['f_full_name'].str.contains("Craw ford@John \(Rev", na=False),'f_last_name'] = 'Crawford'

In [297]:
#Vanderbilt
df.loc[df['f_last_name'].str.contains("Van derbilt", na=False),'f_last_name'] = 'Vanderbilt'

In [298]:
# Patrick
df.loc[df['f_first_name'].str.contains("\[atrick", na=False),'f_first_name'] = 'Patrick'

# Isaac
df.loc[df['f_first_name'].str.contains("\[saac", na=False),'f_first_name'] = 'Isaac'

### Particular Phrases

In [299]:
pattern = '|'.join(['\(See','\(The','\(Kev'])

df['f_first_name'] = df['f_first_name'].astype(str).str.strip().str.replace(pattern, '')
df['f_last_name'] = df['f_last_name'].astype(str).str.strip().str.replace(pattern, '')
df['f_middle_name'] = df['f_middle_name'].astype(str).str.strip().str.replace(pattern, '')

# Handling Specific Phrases

### Handling Initials

In [300]:
# df['f_full_name'] =  df['f_full_name'].str.replace('\s\.','.')
# df['f_full_name'] =  df['f_full_name'].str.replace(' \.\s','.')

In [301]:
# df['f_initials'] = df['f_full_name'].str.findall(r'(\s?[A-Z]\.)+').to_frame()
# #df['f_initials'] = df['f_full_name'].str.extract('(\s?[A-Z]\.)+', expand=False).str.strip()

In [302]:
# import re
# trial= 'Vanpelt P.J. (Rev'
# #print(trial.str.extract('(\s?[A-Z]\s?\.)+', expand=False).str.strip())

# match = re.search(r'(\s?[A-Z]\.)+', trial)

# print(match)  # print time_date >> 0800_20180101
# #print(match.group(2))  # print time >> 0800
# #print(match.group(3))

### Handling certain Titles in Names - 'Capt.', '(Rev)'

1. Moving 'Capt.' to title and move Middle Name to 
2. Moving '(Rev)' to title and 

In [303]:
# Handling 'Capt.' based on above logic
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("Capt", na=False),'f_title'] = 'Capt.'
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("Capt", na=False),'f_first_name'] = df.loc[df['f_full_name'].str.contains("Capt", na=False),'f_middle_name']
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("Capt", na=False),'f_middle_name'] = ''

In [304]:
# Handling 'jr' based on above logic
df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.contains("jr", na=False),'f_title'] = 'jr'
df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.contains("jr", na=False),'f_middle_name'] \
= df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.contains("jr", na=False),'f_middle_name'].str.replace('jr','')

# Handling 'jr' based on above logic
df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.contains("jr", na=False),'f_title'] = 'jr'
df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.contains("jr", na=False),'f_first_name'] \
= df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.contains("jr", na=False),'f_first_name'].str.replace('jr','')

In [305]:
# Handling '(Rev.)' based on above logic
df.loc[df['f_full_name'].astype(str).str.strip().str.contains("\(Rev", na=False),'f_title'] = 'Rev.'
df.loc[df['f_middle_name'].astype(str).str.strip().str.contains("\(Rev", na=False),'f_middle_name'] = df.loc[df['f_middle_name'].str.contains("\(Rev", na=False),'f_middle_name'].str.replace('\(Rev','')
df.loc[df['f_first_name'].astype(str).str.strip().str.contains("\(Rev", na=False),'f_first_name'] = df.loc[df['f_first_name'].str.contains("\(Rev", na=False),'f_first_name'].str.replace('\(Rev','')

In [306]:
####
###
###
##
###
## Seperate Initials in a better way

### Handling 'Mc' in Names 

1. Replacing ' M* ' with 'Mc'
2. Replacing ' M' ' with 'Mc'
2. Replacing ' M'' ' with 'Mc'

In [307]:
# Replacing ' M* ' with 'Mc'
df['f_first_name'] =  df['f_first_name'].str.replace('M\*','Mc')
df['f_last_name'] =  df['f_last_name'].str.replace('M\*','Mc')
df['f_middle_name'] =  df['f_middle_name'].str.replace('M\*','Mc')

In [308]:
# Replacing ' M' ' with 'Mc'
df['f_first_name'] =  df['f_first_name'].str.replace('M\'','Mc')
df['f_last_name'] =  df['f_last_name'].str.replace('M\'','Mc')
df['f_middle_name'] =  df['f_middle_name'].str.replace('M\'','Mc')

In [309]:
# Replacing ' M" ' with 'Mc'
df['f_first_name'] =  df['f_first_name'].str.replace('M\"','Mc')
df['f_last_name'] =  df['f_last_name'].str.replace('M\"','Mc')
df['f_middle_name'] =  df['f_middle_name'].str.replace('M\"','Mc')

In [310]:
# Replacing ' M'' ' with 'Mc'
df['f_first_name'] =  df['f_first_name'].str.replace('M\'\'','Mc')
df['f_last_name'] =  df['f_last_name'].str.replace('M\'\'\'','Mc')
df['f_middle_name'] =  df['f_middle_name'].str.replace('M\'\'','Mc')

In [311]:
# Replacing ' Mc' ' with 'Mc'
df['f_first_name'] =  df['f_first_name'].str.replace('Mc\'','Mc')
df['f_last_name'] =  df['f_last_name'].str.replace('Mc\'','Mc')
df['f_middle_name'] =  df['f_middle_name'].str.replace('Mc\'','Mc')

### Handling 'Van' in Names 

In [312]:
df.loc[df['f_last_name'].astype(str).str.strip().str.lower().str.startswith("van", na=False),'f_last_name'] = df.loc[df['f_last_name'].astype(str).str.strip().str.lower().str.startswith("van", na=False),'f_last_name'].str.replace('Van','Van ')

In [313]:
# Fixing names where 'Van' is in Last_name and the rest is in FN and the acutal FN is in MN
df.loc[df['f_last_name'].astype(str).str.strip().str.lower() == 'van', 'f_working_column']  = df.apply(lambda x:'%s %s' % (x['f_last_name'],x['f_first_name']),axis=1)
df.loc[df['f_last_name'].astype(str).str.strip().str.lower() == 'van', 'f_first_name']  = df['f_middle_name']
df.loc[df['f_last_name'].astype(str).str.strip().str.lower() == 'van', 'f_last_name']  = df['f_working_column']
df.loc[df['f_last_name'].astype(str).str.strip().str.lower() == 'van', 'f_middle_name']  = ''

# Handling Specific Characters

### Removing Numbers from Names

In [314]:
df['f_first_name'] =  df['f_first_name'].str.replace(r'([1-9])+','')
df['f_last_name'] =  df['f_last_name'].str.replace(r'([1-9])+','')
df['f_middle_name'] =  df['f_middle_name'].str.replace(r'([1-9])+','')

### Handling 'â€”â€' in Names

In [315]:
df['f_first_name'] =  df['f_first_name'].str.replace('â€”â€','')
df['f_last_name'] =  df['f_last_name'].str.replace('â€”â€','')
df['f_middle_name'] =  df['f_middle_name'].str.replace('â€”â€','')

### Handling '[' or ']' in Names 

In [316]:
# Finding ']' followed and preceeded by 'l' and replace 
df['f_first_name'] =  df['f_first_name'].str.replace(']l','ll')
df['f_last_name'] =  df['f_last_name'].str.replace(']l','ll')


df['f_first_name'] =  df['f_first_name'].str.replace('l]','ll')
df['f_last_name'] =  df['f_last_name'].str.replace('l]','ll')

# Finding ']' followed and preceeded by 'i' and replace 
df['f_first_name'] =  df['f_first_name'].str.replace(']i','li')
df['f_last_name'] =  df['f_last_name'].str.replace(']i','li')


df['f_first_name'] =  df['f_first_name'].str.replace('i]','il')
df['f_last_name'] =  df['f_last_name'].str.replace('i]','il')


# Finding ']' followed and preceeded by 't' and replace 
df['f_first_name'] =  df['f_first_name'].str.replace(']t','tt')
df['f_last_name'] =  df['f_last_name'].str.replace(']t','tt')


In [317]:
# replacing '[' and ']' in first name
df['f_first_name'] =  df['f_first_name'].str.replace('[','')
df['f_first_name'] =  df['f_first_name'].str.replace(']','')

# replacing '[' and ']' in middle name
df['f_middle_name'] =  df['f_middle_name'].str.replace('[','')
df['f_middle_name'] =  df['f_middle_name'].str.replace(']','')

# replacing '[' and ']' in last name
df['f_last_name'] =  df['f_last_name'].str.replace('[','')
df['f_last_name'] =  df['f_last_name'].str.replace(']','')

### Handling '(' or ')' in Names 

In [318]:
# Handling and replacing '(' in names with ''
df['f_first_name'] =  df['f_first_name'].str.replace('(','')
df['f_last_name'] =  df['f_last_name'].str.replace('(','')
df['f_middle_name'] =  df['f_middle_name'].str.replace('(','')

In [319]:
# Handling and replacing ')' in names with ''
df['f_first_name'] =  df['f_first_name'].str.replace(')','')
df['f_last_name'] =  df['f_last_name'].str.replace(')','')
df['f_middle_name'] =  df['f_middle_name'].str.replace(')','')

### Handling '{' and '}' in names

In [320]:
# Handling and replacing '{' in names with ''
df['f_first_name'] =  df['f_first_name'].str.replace('{','')
df['f_last_name'] =  df['f_last_name'].str.replace('{','')
df['f_middle_name'] =  df['f_middle_name'].str.replace('{','')

In [321]:
# Handling and replacing '}' in names with ''
df['f_first_name'] =  df['f_first_name'].str.replace('}','')
df['f_last_name'] =  df['f_last_name'].str.replace('}','')
df['f_middle_name'] =  df['f_middle_name'].str.replace('}','')

### Handling '!' in Names

In [322]:
df['f_first_name'] =  df['f_first_name'].str.replace('!','')
df['f_last_name'] =  df['f_last_name'].str.replace('!','')
df['f_middle_name'] =  df['f_middle_name'].str.replace('!','')

### Handling ':' in Names

In [323]:
df['f_first_name'] =  df['f_first_name'].str.replace(':','')
df['f_last_name'] =  df['f_last_name'].str.replace(':','')
df['f_middle_name'] =  df['f_middle_name'].str.replace(':','')

### Handling '*' in Names

In [324]:
df['f_first_name'] =  df['f_first_name'].str.replace('*','')
df['f_last_name'] =  df['f_last_name'].str.replace('*','')
df['f_middle_name'] =  df['f_middle_name'].str.replace('*','')

### Handling '.' in Names 

In [325]:
df['f_first_name'] =  df['f_first_name'].str.replace(r'\b.$', '', regex=True).str.strip()
#df['f_last_name'] =  df['f_last_name'].str.replace(r'\b.$', '', regex=True).str.strip()
df['f_middle_name'] =  df['f_middle_name'].str.replace(r'\b.$', '', regex=True).str.strip()

### Handling Special Starting Character in Names 

In [326]:
# replacing special characters if the field value begins with them
df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.startswith(".", na=False),'f_first_name'] \
= df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.startswith(".", na=False),'f_first_name'].str.replace('.','')

df.loc[df['f_last_name'].astype(str).str.strip().str.lower().str.startswith(".", na=False),'f_last_name'] \
= df.loc[df['f_last_name'].astype(str).str.strip().str.lower().str.startswith(".", na=False),'f_last_name'].str.replace('.','')

df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.startswith(".", na=False),'f_middle_name'] \
= df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.startswith(".", na=False),'f_middle_name'].str.replace('.','')

In [327]:
# replacing special characters if the field value begins with them
df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.startswith(",", na=False),'f_first_name'] \
= df.loc[df['f_first_name'].astype(str).str.strip().str.lower().str.startswith(",", na=False),'f_first_name'].str.replace(',','')

df.loc[df['f_last_name'].astype(str).str.strip().str.lower().str.startswith(",", na=False),'f_last_name'] \
= df.loc[df['f_last_name'].astype(str).str.strip().str.lower().str.startswith(",", na=False),'f_last_name'].str.replace(',','')

df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.startswith(",", na=False),'f_middle_name'] \
= df.loc[df['f_middle_name'].astype(str).str.strip().str.lower().str.startswith(",", na=False),'f_middle_name'].str.replace(',','')

### Handling other characters

In [328]:
pattern = '|'.join(['Â»','Â°','â€œBrooklyn', '^','-','\?','@',';', '/','\"', '|', '~', '#','\*','%','_','>','\'','Â', 'â€œo','â€”â€”'])

df['f_first_name'] = df['f_first_name'].astype(str).str.strip().str.replace(pattern, '')
df['f_last_name'] = df['f_last_name'].astype(str).str.strip().str.replace(pattern, '')
df['f_middle_name'] = df['f_middle_name'].astype(str).str.strip().str.replace(pattern, '')

# Final Prep & Flagging

In [329]:
# replacing 'nan' texts that may have creeped in 
df.replace(['None', 'nan'], np.nan, inplace=True)

In [330]:
# Flagging rows that have no First and Last Name
df.loc[df['f_first_name'].isnull() & df['f_last_name'].isnull(), "junk_entries"] = 1

In [331]:
# Creating the  f_full_name name column by concatinating the cleaned up LN,FN,MN
df["f_full_name"] = df.apply(lambda x:'%s %s %s' % (x['f_last_name'],x['f_first_name'], x['f_middle_name']),axis=1)

# Write to BQ

In [332]:
df.to_gbq(destination_table= 'hnyc_exploration.hnyc_working_table', 
          project_id='might-seekers', 
          if_exists='replace', 
          progress_bar=True)

1it [00:11, 11.98s/it]


# Phonetic Encoding of Names using different algorithms

In [ ]:
jellyfish.metaphone(row[in_col])
jellyfish.metaphone(row[in_col])

In [ ]:
def df_soundex(df,in_col, out_col):
  hash_table = collections.defaultdict()
  for index, row in df.iterrows():
    hash_table[index] = jellyfish.soundex(row[in_col])
  
  df.loc[:,out_col] = pd.Series(dict(hash_table),index=df.index)

In [ ]:
def df_metaphone(df,in_col, out_col):
  hash_table = collections.defaultdict()
  for index, row in df.iterrows():
    hash_table[index] = jellyfish.metaphone(row[in_col])
  
  df.loc[:,out_col] = pd.Series(dict(hash_table),index=df.index)

In [ ]:
df['last_name'] = df['last_name'].astype(str)
df['first_name'] = df['first_name'].astype(str)
df['middle_name'] = df['middle_name'].astype(str)

# Get Soundex code for First and Last Name
df_soundex(df,'last_name','last_name_sx')
#df_soundex(df,'first_name','first_name_sx')
#df_soundex(df,'middle_name','middle_name_sx')

# Get Metaphone code for First and Last Name
df_metaphone(df,'last_name','last_name_mph')
#df_metaphone(df,'first_name','first_name_sx')
#df_metaphone(df,'middle_name','middle_name_sx')

In [ ]:
for index, row in df.iterrows():
  comp_val = row[in_col]
  df_slice = df['last_name'][df['last_name_sx'] == comp_val]
  df_slice_list= df_slice.unique()
  jellyfish.levenshtein_distance('Jenny', 'Bunny')
  res = {}
  for key in df_slice_list:
      for value in test_values:
          res[key] = value
          test_values.remove(value)
          break  

In [ ]:
jellyfish.levenshtein_distance('Jenny', 'Bunny')

2

In [ ]:
df_slice = df['last_name'][df['last_name_sx'] == 'A165']
piece= df_slice.unique()

In [ ]:
df_trial['similar_last'] = ''
df_trial.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,full_name,first_name,last_name,middle_name,widow_flag,widow_of,title,occupation,full_address,h_status_flag,h_address,h_house_number,h_street_name,h_city,address_2,address_2_house_number,address_2_street_name,address_2_city,index,full_entry,dataset_id,cd_unique_id,last_name_sx,first_name_sx,similar_last
0,Abberly Richard,Richard,Abberly,NaN,False,NaN,NaN,shoemaker,168 Hudson av,Assume,168 Hudson av,168.0,168 Hudson av,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100001,A164,R263,
1,Abberly Samuel,Samuel,Abberly,NaN,False,NaN,NaN,NaN,mason Dean n Vanderbilt av,Assume,mason Dean n Vanderbilt av,NaN,NaN,"['mason', 'Dean', 'n', 'Vanderbilt', 'av']",NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100002,A164,S540,
2,Abbey A C,A,Abbey,C,False,NaN,NaN,carpenter,82 Poplar,Assume,82 Poplar,82.0,82 Poplar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100003,A100,A000,
3,"Aessy Horatio G , Columbia Institute for the",Horatio,Aessy,G,False,NaN,NaN,NaN,education of,Assume,education of,NaN,NaN,"['education', 'of']",NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100004,A200,H630,
4,Abbey WS,WS,Abbey,NaN,False,NaN,NaN,NaN,merchant N Â¥ h Union n Court,Assume,merchant N Â¥ h Union n Court,NaN,NaN,"['merchant', 'N', 'Â¥', 'h', 'Union', 'n', 'Co...",NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100005,A100,W200,


In [ ]:
df_trial['similar_last'][1]= piece

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_trial.head(5)

,full_name,first_name,last_name,middle_name,widow_flag,widow_of,title,occupation,full_address,h_status_flag,h_address,h_house_number,h_street_name,h_city,address_2,address_2_house_number,address_2_street_name,address_2_city,index,full_entry,dataset_id,cd_unique_id,last_name_sx,first_name_sx,similar_last
0,Abberly Richard,Richard,Abberly,NaN,False,NaN,NaN,shoemaker,168 Hudson av,Assume,168 Hudson av,168.0,168 Hudson av,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100001,A164,R263,
1,Abberly Samuel,Samuel,Abberly,NaN,False,NaN,NaN,NaN,mason Dean n Vanderbilt av,Assume,mason Dean n Vanderbilt av,NaN,NaN,"['mason', 'Dean', 'n', 'Vanderbilt', 'av']",NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100002,A164,S540,"[Aborn, Abraham, Abrahame, Abrahams, Abrams, A..."
2,Abbey A C,A,Abbey,C,False,NaN,NaN,carpenter,82 Poplar,Assume,82 Poplar,82.0,82 Poplar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100003,A100,A000,
3,"Aessy Horatio G , Columbia Institute for the",Horatio,Aessy,G,False,NaN,NaN,NaN,education of,Assume,education of,NaN,NaN,"['education', 'of']",NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100004,A200,H630,
4,Abbey WS,WS,Abbey,NaN,False,NaN,NaN,NaN,merchant N Â¥ h Union n Court,Assume,merchant N Â¥ h Union n Court,NaN,NaN,"['merchant', 'N', 'Â¥', 'h', 'Union', 'n', 'Co...",NaN,NaN,NaN,NaN,NaN,NaN,BK_1850,100005,A100,W200,
